In [1]:
%pylab inline
import datetime
import os
from PIL import Image, ImageDraw
from scipy import ndimage
import cv2
# from sklearn import preprocessing

Populating the interactive namespace from numpy and matplotlib


## Good reference
- [ndimage processing](https://docs.scipy.org/doc/scipy-0.18.1/reference/tutorial/ndimage.html)

In [29]:
def fltr(what, trash):
    return [i for i in what if i not in trash]

def showmed(imgs):
    for _, item in imgs.items():
        imshow(item)
        show()

data = "Lab4/data/"
trash = [".DS_Store"]  # Things I don't want to look at

pics = []
gaudi = {}
rushmore = {}
dame = {}

for i in fltr(os.listdir(data), trash):
    path = os.path.join(data, i)
    if os.path.isdir(path):
        for j in fltr(os.listdir(path),trash):
            pics.append(os.path.join(path,j))

gaudi["a"] = Image.open(pics[0])
gaudi["b"] = Image.open(pics[1])
rushmore["a"] = Image.open(pics[2])
rushmore["b"] = Image.open(pics[3])
dame["a"] = Image.open(pics[4])
dame["b"] = Image.open(pics[5])
img_data = {"gaudi": gaudi, "dame": dame, "rushmore": rushmore}
test = Image.open("Lab4/data/test.png")

In [3]:
if not type(np.array([1])) is numpy.ndarray:
    print ("hello")
elif len(np.array([[[1]]]).shape) > 2:
    print ("reduce")

reduce


## Task 1 (30) Implement Harris corner detector

$$A({\mathbf  {x}})=\sum _{{p,q}}w(p,q){\begin{bmatrix}I_{{x}}^{2}({\mathbf  {x}})&I_{{x}}I_{{y}}({\mathbf  {x}})\\I_{{x}}I_{{y}}({\mathbf  {x}})&I_{{y}}^{2}({\mathbf  {x}})\\\end{bmatrix}}$$

- [Harris Corner Detector @ Penn State University](http://www.cse.psu.edu/~rtc12/CSE486/lecture06.pdf)

In [4]:
def harris(img, α, σI, threshold, save_img=False, show_img=False):
    if not type(img) is numpy.ndarray:
        lum = np.array(img.convert("YCbCr"))[:,:,0]
    elif len(img.shape) > 2:
        lum = img[:,:,0]
    (dimy, dimx) = lum.shape
    
    Ix = np.zeros((dimy, dimx))
    Iy = Ix.copy()
    ndimage.gaussian_filter(input=lum, sigma=[0,1], order=1, output=Ix)
    ndimage.gaussian_filter(input=lum, sigma=[1,0], order=1, output=Iy)

    Ix2 = Ix*Ix
    Iy2 = Iy*Iy
    Ixy = Ix*Iy

    ndimage.gaussian_filter(input=Ix2, sigma=σI, output=Ix2, order=0)
    ndimage.gaussian_filter(input=Iy2, sigma=σI, output=Iy2, order=0)
    ndimage.gaussian_filter(input=Ixy, sigma=σI, output=Ixy, order=0)

    R = ((Ix2*Iy2) - (Ixy*Ixy)) - α* ((Ix2+Iy2)**2)
            
    notCornersIndices = R < threshold
    R[notCornersIndices] = 0
    
    if show_img:
        for i in [Ix,Iy,Ix2,Iy2,Ixy,R]:
            imshow(i)
            show()
        
    if save_img:
        Image.fromarray((Ix+127).astype(np.uint8)).save("Lab4/Ix.png")
        Image.fromarray((Iy+127).astype(np.uint8)).save("Lab4/Iy.png")
        Image.fromarray((Ix2+127).astype(np.uint8)).save("Lab4/gIx2.png")
        Image.fromarray((Iy2+127).astype(np.uint8)).save("Lab4/gIy2.png")
        Image.fromarray((Ixy+127).astype(np.uint8)).save("Lab4/gIxy.png")
        Image.fromarray((R+127).astype(uint8)).save("Lab4/har.png")
    
    return R   

def nms(R, non_max_size=8):
    filterR = ndimage.maximum_filter(input=R, size=non_max_size) != R
    R[filterR] = 0
    return R

In [5]:
sigma = 3
threshold = 7000
k = 0.05 # optimal 0.04 - 0.06
nmsize = 8 # non-maxima suppression filter size
picture = test.copy().convert()

# (img, k, s, threshold, non_max_size
t0 = datetime.datetime.now()
R = harris(picture, k, sigma, threshold)
# R[R < threshold] = 0
R = nms(R, nmsize)
d1,d2 = np.nonzero(R)
print("Harris: "+str(datetime.datetime.now()-t0))
print("Done! Found: "+str(len(d1)))

drw = ImageDraw.Draw(picture)
for i,j in zip(d1,d2):
    drw.ellipse([(j-3,i-3),(j+3,i+3)],outline=(0,255,0))
           
del drw
picture.save("Lab4/t01_test_th"+str(threshold)+"_s"+str(sigma)
             +"_k"+str(k)+"_nms"+str(nmsize)+".png")

Harris: 0:00:00.201381
Done! Found: 62


In [6]:
img_key = 'a'
name = "dame"
sigma = 3
threshold = 5000
k = 0.05 # optimal 0.04 - 0.06
nmsize = 8 # non-maxima suppression filter size
picture = img_data[name][img_key].copy()

# (img, k, s, threshold, non_max_size)
R = harris(picture, k, sigma, threshold)
# R[R < threshold] = 0
R = nms(R, nmsize)
d1,d2 = np.nonzero(R)
print("Done! Found: "+str(len(d1)))

drw = ImageDraw.Draw(picture)
for i,j in zip(d1,d2):
    drw.ellipse([(j-3,i-3),(j+3,i+3)],outline=(0,255,0))
           
del drw
picture.save("Lab4/t01_"+name+"_"+img_key
             +"_th"+str(threshold)+"_s"+str(sigma)
             +"_k"+str(k)+"_nms"+str(nmsize)+".png")

Done! Found: 2301


## Task 2 (30) Adaptive Non-maximal Suppression.

In [7]:
from scipy.spatial.distance import cdist

def anms(R, numPts = 500):
    d1,d2 = np.nonzero(R)
    pts = R[d1,d2]
    tplist = list(zip(pts,d1,d2))
    tplist.sort(reverse=True)
    corners = np.array(tplist)   # easier to index than vanilla array of tuples
    clen,_ = corners.shape
    rpoints = [(float('inf'), tuple(corners[0]), tuple(corners[0]))]
    
    for i in range(1,clen):
        rs = cdist([corners[i,1:]],corners[:i,1:]).flatten()
        ind = np.argmin(rs)
        r = rs[ind]
        rpoints.append((r, tuple(corners[i]), tuple(corners[ind])))
        
    rpoints.sort(reverse=True)
    return rpoints[:numPts]
        

In [8]:
img_key = 'a'
name = "dame"
sigma = 3
threshold = 5000
k = 0.05 # optimal 0.04 - 0.06
nmsize = 8 # non-maxima suppression filter size
picture = img_data[name][img_key].copy()

R = harris(picture, k, sigma, threshold)
# R[R < threshold] = 0
R = nms(R, nmsize)
points = anms(R, numPts=300)
print("Done! Found: "+str(len(points)))

drw = ImageDraw.Draw(picture)
for (r,(_,i,j),(_,a,b)) in points:
#     drw.ellipse([(j-r,i-r),(j+r,i+r)],outline=(255,0,0))
#     drw.line([j,i,b,a],fill=(0,0,255),width=1)
    drw.ellipse([(j-3,i-3),(j+3,i+3)],outline=(0,255,0))
           
del drw
picture.save("Lab4/t02_"+name+"_"+img_key
             +"_th"+str(threshold)+"_s"+str(sigma)
             +"_k"+str(k)+"_nms"+str(nmsize)+".png")

Done! Found: 300


## Task 3 (40) Scale Invariant Keypoint Detector.

### a) Harris-Laplace
- [Richard J Radke, _Computer vision for visual effects_, page 114-117](https://cvfxbook.com/about/)

In [19]:
def harris_laplace(img, α, threshold):
    lum = np.array(img.convert("YCbCr"))[:,:,0]
    # SCALE PARAMETERS
    σ0 = 1
    k  = 1.2
    num_steps = 13
    ks = k**np.array(range(num_steps))
    
    σI = ks*σ0  #sigmas for all different scales

    σD = 0.7 * σI  #sigmas for derivatives
    har = []
    lap = []
    for ak, sigI, sigD in zip(ks, σI, σD):
        scaled = ndimage.gaussian_filter(input=img, sigma=sigD, order=0)
        H = ak**2 * harris(scaled, α, sigI, 0)
        H[H < threshold] = 0
        H = nms(H)
        L = ndimage.gaussian_filter(input=lum, sigma=sigD, order=2)
        har.append(H)
        lap.append(L)
    
    har = np.array(har)
    co = 0;
#     for i in har:
#         cv2.imwrite("har_"+str(co)+".png",i)
#         co += 1
    
    lap = np.array(lap)
#   finding local maximum in Laplacians
    maxes = ndimage.maximum_filter(input=lap, size=(3,0,0)) == lap
    max_scale_ind = np.argmax(maxes.view(dtype=np.int8), axis=0)
    ys,xs = max_scale_ind.shape
    points = []
    for j in range(ys):
        for i in range(xs):
            scl = max_scale_ind[j,i]
            if har[scl, j, i] > 0:
                points.append((ks[scl],j, i))

    return points

In [20]:
img_key = 'a'
name = "test"
sigma = 3
threshold = 5000
α = 0.05 # optimal 0.04 - 0.06
nmsize = 8 # non-maxima suppression filter size
picture = test.copy()
points = harris_laplace(picture,α,threshold)

In [21]:
print("Done! Found: "+str(len(points)))
drw = ImageDraw.Draw(picture)
for (scl,j,i) in points:
    drw.ellipse([(i-scl,j-scl),(i+scl,j+scl)],outline=(255,0,0))
    drw.line([i,j,i+scl,j+scl],fill=(0,0,255),width=1)
    drw.ellipse([(i-3,j-3),(i+3,j+3)],outline=(0,255,0))
           
del drw
picture.save("Lab4/t03_"+name+"_"+img_key
             +"_th"+str(threshold)+"_s"+str(sigma)
             +"_k"+str(k)+"_nms"+str(nmsize)+".png")




Done! Found: 49


In [38]:
# img_key = 'a'
# name = "gaudi"
def comp_points(name, img_key):
    sigma = 3
    threshold = 5000
    α = 0.05 # optimal 0.04 - 0.06
    picture = img_data[name][img_key].copy()
    pic_name = name+"_"+img_key+"_th"+str(threshold)+"_a"+str(α)

    points = harris_laplace(picture,α,threshold)
    print("A "+pic_name + " done! Found: "+str(len(points)))
    # Saving to a file
    to_save = np.array(points)
    np.save("Lab4/points/"+pic_name, to_save, allow_pickle=False)


In [39]:
keys = ['a','b']
names = ["gaudi", "dame", "rushmore"]
for n, k in [(nn,kk) for nn in names for kk in keys]:
    comp_points(n,k)

A gaudi_a_th5000_a0.05 done! Found: 11623
A gaudi_b_th5000_a0.05 done! Found: 1101
A dame_a_th5000_a0.05 done! Found: 3143
A dame_b_th5000_a0.05 done! Found: 2722
A rushmore_a_th5000_a0.05 done! Found: 742
A rushmore_b_th5000_a0.05 done! Found: 4845


In [27]:
points = np.load("Lab4/points/"+pic_name+".npy")

drw = ImageDraw.Draw(picture)
for (scl,j,i) in points:
    drw.ellipse([(i-scl,j-scl),(i+scl,j+scl)],outline=(255,0,0))
    drw.line([i,j,i+scl,j+scl],fill=(0,0,255),width=1)
    drw.ellipse([(i-3,j-3),(i+3,j+3)],outline=(0,255,0))
           
del drw

picture.save("Lab4/t03_"+pic_name+".png")

array([[  1.20000000e+00,   5.00000000e+00,   1.21000000e+03],
       [  1.20000000e+00,   2.40000000e+01,   1.22900000e+03],
       [  1.00000000e+00,   3.20000000e+01,   1.21200000e+03],
       ..., 
       [  1.00000000e+00,   1.79800000e+03,   2.99000000e+02],
       [  1.00000000e+00,   1.79900000e+03,   1.34500000e+03],
       [  1.72800000e+00,   1.80000000e+03,   1.34500000e+03]])

### b) Difference of Gaussians (DoG)